In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import category_encoders as ce
import functools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Conv1D, Embedding,Reshape, Concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras import metrics
from keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction import FeatureHasher

'\nfrom keras.models import Sequential, Model\nfrom keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Conv1D, Embedding,Reshape, Concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D\nfrom keras.layers.merge import Concatenate\nfrom keras.preprocessing import sequence\nfrom keras.optimizers import RMSprop, Adam\nfrom keras.preprocessing.sequence import pad_sequences\nfrom keras.utils import to_categorical\nfrom keras.utils import plot_model\nfrom keras import backend as K\nfrom keras.wrappers.scikit_learn import KerasClassifier\nfrom keras import metrics\nfrom keras import regularizers\n\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import OneHotEncoder\nfrom sklearn.preprocessing import LabelBinarizer\nfrom sklearn import preprocessing\nfrom sklearn.model_selection import GridSearchCV \nfrom sklearn.metrics import classification_report\nfrom sklearn.preprocessing import OrdinalEncoder\nfrom sklearn.feature_extracti

# Data Reading

In [5]:
data = pd.read_csv('./../../full_RASFF_DATA.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
df = data
df

,NUMBER,CLASSIF,DATE_CASE,REF,NOT_COUNTRY,SUBJET,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,PRODUCT,HAZARDS,HAZARDS_CAT,ORIG_COUNTRY,DESTIN_COUNTRY,CONCERN_COUNTRY
15795,12925.0,border rejection,2016-01-29,2016.AET,United Kingdom,aflatoxins (B1 = 31 µg/kg - ppb) in chopped pe...,pet food,feed,serious,destruction,product not (yet) placed on the market,chopped peanuts,aflatoxins,mycotoxins,China,,United Kingdom
35674,32872.0,border rejection,2009-12-16,2009.CDD,Netherlands,aflatoxins (B1 = 4.9; Tot. = 6.0 / B1 = 6.3; T...,"nuts, nut products and seeds",food,undecided,(obsolete),no distribution,peanuts,aflatoxins,mycotoxins,United States,,"Netherlands,Spain"
45949,43171.0,information,2006-08-01,2006.BTK,United Kingdom,unauthorised irradiation of ginseng root powde...,"dietetic foods, food supplements, fortified foods",food,undecided,seizure,distribution restricted to notifying country,ginseng root powder,unauthorised irradiation,radiation,United States,"Germany,United Kingdom",
41404,38602.0,information,2008-02-15,2008.0180,Denmark,Salmonella typhimurium (DT NT Multiresistant -...,poultry meat and poultry meat products,food,undecided,recall from consumers,product past use-by date,turkey breast fillets,salmonella enterica ser. typhimurium,pathogenic micro-organisms,Poland,"Denmark,Germany,Netherlands",
55057,52266.0,information,2003-03-28,2003.ASK,Germany,aflatoxins in blending of nut with shell,"nuts, nut products and seeds",food,undecided,import not authorised,,blending of nut with shell,aflatoxins,mycotoxins,Iran,,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56053,53267.0,information,2002-09-18,2002.BDI,Italy,incorrect labelling on Foodstuffs intended for...,"dietetic foods, food supplements, fortified foods",food,undecided,import not authorised,,foodstuffs intended for special nutritional uses,incorrect labelling,labelling absent/incomplete/incorrect,United States,,Italy
30588,27732.0,alert,2011-06-30,2011.0865,Belgium,shigatoxin-producing Escherichia coli (O157:H7...,meat and meat products (other than poultry),food,undecided,official detention,distribution to other member countries,bovine meat,shigatoxin-producing escherichia coli,microbial contaminants (other),"Belgium,France","Belgium,Greece,Netherlands",
56675,53890.0,information,2002-04-17,2002.AKY,Greece,"Listeria monocytogenes (n=5, c=5) in poultry m...",poultry meat and poultry meat products,food,undecided,import not authorised,,poultry meat preparations,listeria monocytogenes,microbial contaminants (other),Brazil,,"Greece,Spain"
19406,16549.0,information for attention,2014-11-13,2014.1534,Slovenia,mercury (0.22 mg/kg - ppm) in food supplement ...,"dietetic foods, food supplements, fortified foods",food,serious,official detention,distribution restricted to notifying country,food supplement,mercury,metals,China,Slovenia,


In [4]:
mask = (df['DATE_CASE'] >= '2004-01-01') & (df['DATE_CASE'] <= '2018-12-31')
df = df.loc[mask]
df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
df['DATE_CASE'] = df.DATE_CASE.dt.month

L:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
L:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [318]:
features = [0,1,2,6,7,8]
target = [5]
X = df.iloc[:,features]
y = df.iloc[:,target]

In [319]:
X

,DATE_CASE,NOT_COUNTRY,PROD_CAT,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN
38473,1,Germany,cephalopods and products thereof,distribution to other member countries,metals,India
81393,9,Netherlands,herbs and spices,information on distribution not (yet) available,pathogenic micro-organisms,Germany
98111,10,Netherlands,"nuts, nut products and seeds",no distribution,mycotoxins,Brazil
104115,12,Netherlands,"nuts, nut products and seeds",no distribution,mycotoxins,Egypt
107327,8,Italy,cereals and bakery products,distribution on the market (possible),foreign bodies,France
131374,11,Slovenia,animal nutrition - (obsolete),distribution on the market (possible),pathogenic micro-organisms,Bosnia and Herzegovina
114796,8,Italy,fish and fish products,no distribution,poor or insufficient controls,Croatia
73701,7,United Kingdom,fruits and vegetables,product (presumably) no longer on the market,composition,Costa Rica
93018,6,Poland,cereals and bakery products,no distribution,foreign bodies,Ukraine
66580,4,France,"dietetic foods, food supplements, fortified foods",distribution to other member countries,composition,United Kingdom


In [320]:
y

,ACTION_TAKEN
38473,seizure
81393,re-dispatch
98111,(obsolete)
104115,official detention
107327,seizure
131374,
114796,re-dispatch
73701,informing authorities
93018,re-dispatch
66580,recall from consumers


# Codification Selection

In [321]:
ency = OneHotEncoder(handle_unknown='ignore', sparse = False)
ency.fit(y.values)
y_one_hot = ency.transform(y.values)

In [322]:
'''encx = OrdinalEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

'encx = OrdinalEncoder()\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)'

In [323]:
'''encx = OneHotEncoder(handle_unknown='ignore', sparse = False)
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

"encx = OneHotEncoder(handle_unknown='ignore', sparse = False)\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)"

In [324]:
encx  = ce.BaseNEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)

In [326]:
X_train, X_val, y_train, y_val = train_test_split(x_one_hot, y_one_hot, test_size=0.2)

# Basic mlp Test

In [329]:
accs = []
for i in range (1,6):
    K.clear_session()
    model = Sequential()
    model.add(Dense(2048, activation="relu", input_dim=len(X_train.values[1])))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(24, activation = "softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam",  metrics=['categorical_accuracy'])
    hist = model.fit(X_train,y_train, epochs= 15,validation_data=(X_val, y_val), batch_size = 64)
    accs.append(hist.history['val_categorical_accuracy'][9])
    model.save ("model"+str(i)+".h5")

Train on 105355 samples, validate on 26339 samples
Epoch 1/15
105355/105355 [==============================] - 8s 72us/step - loss: 1.7786 - categorical_accuracy: 0.4564 - val_loss: 1.4485 - val_categorical_accuracy: 0.5461
Epoch 2/15
105355/105355 [==============================] - 7s 68us/step - loss: 1.3853 - categorical_accuracy: 0.5702 - val_loss: 1.2360 - val_categorical_accuracy: 0.6137
Epoch 3/15
105355/105355 [==============================] - 7s 68us/step - loss: 1.2092 - categorical_accuracy: 0.6202 - val_loss: 1.1074 - val_categorical_accuracy: 0.6480
Epoch 4/15
105355/105355 [==============================] - 7s 68us/step - loss: 1.0989 - categorical_accuracy: 0.6521 - val_loss: 1.0380 - val_categorical_accuracy: 0.6712
Epoch 5/15
105355/105355 [==============================] - 7s 69us/step - loss: 1.0197 - categorical_accuracy: 0.6716 - val_loss: 0.9857 - val_categorical_accuracy: 0.6860
Epoch 6/15
105355/105355 [==============================] - 7s 68us/step - loss: 0.9

In [349]:
suma = 0
for i in accs:
    suma = suma + i
print(suma/5)

0.7319564144768591
